In [ ]:
import numpy

In [ ]:
def velocity_term(b, rho, dt, u, v, dx):
    b[1:-1, 1:-1] = (
        rho * dx / 16 * 
        (2 / dt * (u[1:-1, 2:] - 
                   u[1:-1, :-2] + 
                   v[2:, 1:-1] - 
                   v[:-2, 1:-1]) - 
         2 / dx * (u[2:, 1:-1] - u[:-2, 1:-1]) * 
                  (v[1:-1, 2:] - v[1:-1, :-2]) -
        (u[1:-1, 2:] - u[1:-1, :-2])**2 / dx -
        (v[2:, 1:-1] - v[:-2, 1:-1])**2 / dx) 
                    )

    return b

In [ ]:
def pressure_poisson(p, b, l2_target=1e-3):
    iter_diff = l2_target + 1
    while iter_diff > l2_target:
        n = 0
        pn = p.copy()
        p[1:-1,1:-1] = (.25 * (pn[1:-1, 2:] +
                               pn[1:-1, :-2] +
                               pn[2:, 1:-1] +
                               pn[:-2, 1:-1]) -
                               b[1:-1, 1:-1])
        
        p[:, 0] = p[:, 1]   #dp/dx = 0 at x = 0
        p[:, -1] = p[:, -2] #dp/dy = 0 at x = 2
        p[0, :] = p[1, :]   #dp/dy = 0 at y = 0
        p[-1, :] = 0        #p = 0 at y = 2      
        
        if n % 10 == 0:
            iter_diff = numpy.sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2))
        
        if n == 500:
            break
            
        n += 1
        
    return p

In [ ]:
from snippets.ns_helper import cavity_flow

In the interests of brevity, we're only going to worry about the pressure poisson solver.  There rest of the 2D Navier-Stokes solution is encapsulated in the function `cavity_flow`

In [ ]:
import pickle

In [ ]:
def run_cavity():
    nx = 41
    ny = 41
    with open('IC.pickle', 'rb') as f:
        u, v, p, b = pickle.load(f)

    dx = 2 / (nx - 1)
    dy = 2 / (ny - 1)
    dt = .005
    nt = 100
    
    u, v, p = cavity_flow(u, v, p, nt, dt, dx, 
                         velocity_term, 
                         pressure_poisson)
    
    return u, v, p

So what does this all do?  Let's check it out.

In [ ]:
u, v, p = run_cavity()

In [ ]:
%matplotlib inline
from snippets.ns_helper import quiver_plot

In [ ]:
quiver_plot(u, v, p)

Let's profile the `cavity_flow` function and see if there's a specific place that's really hurting our performance.

In [ ]:
%timeit run_cavity()

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f cavity_flow run_cavity()

## Where is the bottleneck?

Clearly the PPE is the problem here, so let's use `numba` to rewrite it.  

```python
@jit(nopython=True)
def pressure_poisson(p, b, l2_target=1e-3):
    J, I = b.shape
        
    iter_diff = l2_target + 1
    
    while iter_diff > l2_target:
        n = 0
        pn = p.copy()
        
        
        if n % 10 == 0:
            iter_diff = numpy.sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2))
            
        if n == 500:
            break
            
        n += 1
        
    return p
    ```

### Reference:  
The equation we need to unroll is given by 

\begin{equation}
p_{i,j}^{n} = \frac{1}{4}\left(p_{i+1,j}^{n}+p_{i-1,j}^{n}+p_{i,j+1}^{n}+p_{i,j-1}^{n}\right) - b
\end{equation}

and recall that `b` is already computed, so no need to worry about unrolling that.

In [ ]:
from numba import jit

In [ ]:
%load snippets/ppe_numba.py

In [ ]:
un, vn, pn = run_cavity()

In [ ]:
assert numpy.allclose(p, pn)
assert numpy.allclose(u, un)
assert numpy.allclose(v, vn)

In [ ]:
%timeit run_cavity()

In [ ]:
%load_ext cython

In [ ]:
%load snippets/ppe_cython.py

In [ ]:
%timeit run_cavity()